In [30]:
import fitz
import re
import os
import tiktoken

Documentation of RAG project



## Parsing
For the baseline a simple parser (fitz/PyMuPDF) was selected. It does the bare minimum.

### No normalization
First we tried with no normalization which looks like this:

In [31]:
def parse_document(pdf_path):
    doc = fitz.open(pdf_path)
    text_and_pagenumber = []  # List [(page_number, page_text)]

    for i, page in enumerate(doc):
        text = page.get_text(sort=True)
        if text.strip():  # Skip empty pages
            text_and_pagenumber.append((i + 1, text))
    doc.close()
    return text_and_pagenumber

text_and_pagenumber = parse_document("../../pdf_data/12_BERÄTTELSER_OM_SKAM.pdf")

In [32]:
text_and_pagenumber[0:5]

[(1,
  '12 BERÄTTELSER OM\n      SKAM\n\n En kvalitativ studie om terapeuters\nupplevelser och erfarenheter av egen\n      skam i terapirummet\n\n              Karin Forsberg och Malin Lundin\n\n\n\n\n\n                            Examensarbete, 15 hp\n\n                         Psykoterapeutprogrammet, 90 hp\n\n                                    Ht 2024\n\n                            Handledare: Per Höglund'),
 (2,
  '                                      Tack!\n\nEtt stort tack till våra informanter för att ni med lust och engagemang tog er tid till att dela era\nupplevelser och erfarenheter med oss. Utan er hade det inte blivit någon uppsats! Vi riktar även\nett varmt tack till vår handledare Per Höglund som med stor entusiasm och inspiration guidat\noss igenom uppsatsskrivandet. Slutligen vill vi med värme tacka Thomas och Fredrik för\nförståelse och stöd samt till våra barn som betyder allt!'),
 (3,
  "                                                             1\n\n\n          

### With normalization
That was not optimal, so we tried with normalization:

In [33]:
def normalize_text(input_text):
    # Remove split words at the end of lines
    normalized = re.sub(r"- ?\n", "", input_text.strip())
    # Replace any sequence of whitespace (including newlines) with a single space
    normalized = re.sub(r"\s+", " ", normalized)
    # Don't keep space if end of sentence
    normalized = re.sub(r" +\.\s", ". ", normalized)

    return normalized

def parse_document_norm(pdf_path):
    doc = fitz.open(pdf_path)
    text_and_pagenumber = []  # List [(page_number, page_text)]

    for i, page in enumerate(doc):
        text = page.get_text(sort=True)
        if text.strip():  # Skip empty pages
            norm_text = normalize_text(text)
            text_and_pagenumber.append((i + 1, norm_text + " "))
    doc.close()
    return text_and_pagenumber

text_and_pagenumber_norm = parse_document_norm("../../pdf_data/12_BERÄTTELSER_OM_SKAM.pdf")

In [34]:
text_and_pagenumber_norm[0:5]

[(1,
  '12 BERÄTTELSER OM SKAM En kvalitativ studie om terapeuters upplevelser och erfarenheter av egen skam i terapirummet Karin Forsberg och Malin Lundin Examensarbete, 15 hp Psykoterapeutprogrammet, 90 hp Ht 2024 Handledare: Per Höglund '),
 (2,
  'Tack! Ett stort tack till våra informanter för att ni med lust och engagemang tog er tid till att dela era upplevelser och erfarenheter med oss. Utan er hade det inte blivit någon uppsats! Vi riktar även ett varmt tack till vår handledare Per Höglund som med stor entusiasm och inspiration guidat oss igenom uppsatsskrivandet. Slutligen vill vi med värme tacka Thomas och Fredrik för förståelse och stöd samt till våra barn som betyder allt! '),
 (3,
  "1 Sammanfattning Denna studie undersöker terapeuters egna erfarenheter och upplevelser av skam i terapirummet och hur det påverkar den terapeutiska alliansen och maktbalansen i relationen till klienten. Tidigare forskning har främst fokuserat på klientens skam, medan terapeutens egna skamupple

## Tokenizing & Chunking

tiktoken was used for tokenizing. A hard limit of 512 tokens per chunk was set.
Chunk text with metadata are saved in dictionary:

In [35]:
EMBEDDING_MODEL_NAME = "text-embedding-3-small"
TOKEN_ENCODER = tiktoken.encoding_for_model(EMBEDDING_MODEL_NAME)

def chunk_pdf_by_tokens(pdf_path, MAX_TOKENS=512):
    filename = os.path.basename(pdf_path)
    text_and_pagenumber = parse_document_norm(pdf_path)  # List [(page_number, page_text)]
    chunks = []
    all_tokens = []
    token_page_map = []
    for page_number, page_text in text_and_pagenumber:
        tokens = TOKEN_ENCODER.encode(page_text)
        all_tokens.extend(tokens)
        token_page_map.extend([page_number] * len(tokens))

    # Split into chunks of MAX_TOKENS
    total_chunks = (len(all_tokens) + MAX_TOKENS - 1) // MAX_TOKENS

    for i in range(total_chunks):
        start = i * MAX_TOKENS
        end = start + MAX_TOKENS
        token_chunk = all_tokens[start:end]
        chunk_text = TOKEN_ENCODER.decode(token_chunk)

        chunk_pages = token_page_map[start:end]
        page_list = sorted(set(chunk_pages))

        chunk_metadata = {
            "id": f"{filename}_chunk{i + 1}",
            "filename": filename,
            "page_number": ",".join(map(str, page_list)),
            "chunk_index": i + 1,
            "total_chunks": total_chunks,
        }

        chunks.append({"text": chunk_text, "metadata": chunk_metadata})
    return chunks

c = chunk_pdf_by_tokens("../../pdf_data/12_BERÄTTELSER_OM_SKAM.pdf")

In [36]:
c[0:5]

[{'text': '12 BERÄTTELSER OM SKAM En kvalitativ studie om terapeuters upplevelser och erfarenheter av egen skam i terapirummet Karin Forsberg och Malin Lundin Examensarbete, 15 hp Psykoterapeutprogrammet, 90 hp Ht 2024 Handledare: Per Höglund Tack! Ett stort tack till våra informanter för att ni med lust och engagemang tog er tid till att dela era upplevelser och erfarenheter med oss. Utan er hade det inte blivit någon uppsats! Vi riktar även ett varmt tack till vår handledare Per Höglund som med stor entusiasm och inspiration guidat oss igenom uppsatsskrivandet. Slutligen vill vi med värme tacka Thomas och Fredrik för förståelse och stöd samt till våra barn som betyder allt! 1 Sammanfattning Denna studie undersöker terapeuters egna erfarenheter och upplevelser av skam i terapirummet och hur det påverkar den terapeutiska alliansen och maktbalansen i relationen till klienten. Tidigare forskning har främst fokuserat på klientens skam, medan terapeutens egna skamupplevelser har varit rela

## Embedding Documents

The embedding model "text-embedding-3-small" was used for the embeddings.

This is how they were created and inserted to chromadb:

```python
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key=OPENAI_KEY, model_name=EMBEDDING_MODEL_NAME
)
chroma_client = chromadb.PersistentClient(path=PERSIST_DIRECTORY)
collection = chroma_client.get_or_create_collection(
    name=COLLECTION_NAME, embedding_function=openai_ef
)
client = OpenAI(api_key=OPENAI_KEY)

def process_pdfs_and_insert(directory):
    for filename in os.listdir(directory):
        if filename.endswith(".pdf"):
            pdf_path = os.path.join(directory, filename)
            chunks = chunk_pdf_by_tokens(pdf_path)

            for chunk in chunks:
                chunk_id = chunk["metadata"]["id"]
                chunk_text = chunk["text"]
                embedding = (
                    client.embeddings.create(
                        input=chunk_text, model=EMBEDDING_MODEL_NAME
                    )
                    .data[0]
                    .embedding
                )
                collection.upsert(
                    ids=[chunk_id],
                    documents=[chunk_text],
                    embeddings=[embedding],
                    metadatas=[chunk["metadata"]],
                )
```

## Embedding questions

We needed to create embeddings for our questions. Same embedding model as for the documents.

We open our toml files, parse them, and add a new key for the embeddings.

Then we write a new toml file, which includes the embedding

```python
def get_embedding(question):
    return client.embeddings.create(input=question, model=EMBEDDING_MODEL_NAME).data[0].embedding

def add_embeddings_to_toml(toml_dir):
    for filename in os.listdir(toml_dir):
        if filename.endswith(".toml"):
            with open(toml_dir + filename, "r", encoding="utf-8") as f:
                toml_f = parse(f.read())
            for question in toml_f["questions"]:
                question["question_embedding"] = get_embedding(question["question"])
            with open(toml_dir + "embedded_" + filename, "w", encoding="utf-8") as f:
                f.write(dumps(toml_f))
```

#### This is the before-and-after for a question in the toml file.

```toml
# Before embedding
[[questions]]
id = "PMCCPPN003"
question = "Vilka är de tre utgåvorna av Visual Studio?"
answer = "Det finns tre utgåvor av Visual Studio: Community, Professional och Enterprise"
difficulty = "Easy"
category = "Programming"

[[questions.files]]
file = "cpp-get-started-msvc-170-001.pdf"
page_numbers = [9]

# After embedding
[[questions]]
id = "PMCCPPN003"
question = "Vilka är de tre utgåvorna av Visual Studio?"
answer = "Det finns tre utgåvor av Visual Studio: Community, Professional och Enterprise"
difficulty = "Easy"
category = "Programming"
question_embedding = [-0.06469225883483887, -0.0018141228938475251, 0.03967900574207306, ...]

[[questions.files]]
file = "cpp-get-started-msvc-170-001.pdf"
page_numbers = [9]
```